In [3]:
import requests
from bs4 import BeautifulSoup
import time
from openpyxl import load_workbook, Workbook
from datetime import datetime
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
import pandas as pd 
import tensorflow as tf 

# Load the data from the Excel file
data = pd.read_excel(r"D:\\All_Coding_stuff\\FinancialDashboard\\Model.xlsx")

# Convert the 'Date' column to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Normalize the 'Close' column
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_close = scaler.fit_transform(data[['Close']])

# Define window size
window_size = 10 # Adjust this as needed

# Function to create sequences
def create_sequences(data, window_size):
    X, y = [], []
    for i in range(len(data) - window_size):
        X.append(data[i:i+window_size])
        y.append(data[i+window_size])
    return np.array(X), np.array(y)

X, y = create_sequences(scaled_close, window_size)

# Split the data into training and testing sets
split = int(0.8 * len(X))
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the LSTM model

# Define the model
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(50, activation='relu', return_sequences=True, input_shape=(window_size, 1)),
    tf.keras.layers.LSTM(50, activation='relu'),
    tf.keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mse')
# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32, verbose=1)

# Evaluate the model
loss = model.evaluate(X_test, y_test)
print("Test Loss:", loss)

# Predict the next 5 prices
current_price = scaled_close[-window_size:]
next_5_prices = []

for _ in range(5):
    X_pred = np.array([current_price])
    X_pred = np.reshape(X_pred, (1, window_size, 1))
    predicted_price = model.predict(X_pred)[0][0]
    next_5_prices.append(predicted_price)
    current_price = np.append(current_price[1:], predicted_price)

print("Predicted Prices for the next 5 seconds:", next_5_prices)


Epoch 1/100
178/178 [==============================] - 4s 6ms/step - loss: 8.3069e-04
Epoch 2/100
178/178 [==============================] - 1s 6ms/step - loss: 2.0965e-05
Epoch 3/100
178/178 [==============================] - 1s 6ms/step - loss: 2.2593e-05
Epoch 4/100
178/178 [==============================] - 1s 5ms/step - loss: 2.3401e-05
Epoch 5/100
178/178 [==============================] - 1s 6ms/step - loss: 2.0485e-05
Epoch 6/100
178/178 [==============================] - 1s 6ms/step - loss: 2.1351e-05
Epoch 7/100
178/178 [==============================] - 1s 7ms/step - loss: 2.0905e-05
Epoch 8/100
178/178 [==============================] - 1s 5ms/step - loss: 2.1103e-05
Epoch 9/100
178/178 [==============================] - 1s 5ms/step - loss: 2.0222e-05
Epoch 10/100
178/178 [==============================] - 1s 6ms/step - loss: 2.0699e-05
Epoch 11/100
178/178 [==============================] - 1s 8ms/step - loss: 1.8909e-05
Epoch 12/100
178/178 [==============================

In [2]:
# Save the model in ".h5" format
model.save("D:\\All_Coding_stuff\\FinancialDashboard\\Model\\model.h5")


d:\coding_everything\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
